# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [1]:
import random

class Base:
    # Logging levels
    SILENT = 6
    DEBUG = 1
    INFO = 2
    WARNING = 3
    ERROR = 4
    CRITICAL = 5
    
    def __init__(self, level=INFO):
        self.level = level
        
    def message(self, level, *args):
        if level >= self.level:
            print(*args)

class Card(Base):
    __suits = ["Clubs", "Diamonds", "Hearts", "Spades", "ShuffleCard"]
    __values = list(range(2, 11)) + ["Jack", "Queen", "King", "Ace"]

    def __init__(self, suit, value=None):
        Base.__init__(self)
        self.__suit = suit if suit in self.__suits else None
        self.__value = value if value in self.__values or suit == "ShuffleCard" else None
        
        if self.__suit is None:
            self.message(self.ERROR, "Error, bad suit:", suit)

        if self.__value is None and self.__suit != "ShuffleCard":
            self.message(self.ERROR, "Error, bad value:", value)

    def value(self):
        return self.__value
    
    def suit(self):
        return self.__suit
    
    def numerical_value(self):
        # Special Handling of aces - they can be 1 or 11
        # Here we return 1, and handle the Ace as 11 in the hand evaluation
        if self.__value == "Ace":
            return 1
        elif self.__value in ["Jack", "Queen", "King"]:
            return 10
        else:
            return self.__value
        
    def count_value(self):
        """Return the card counting value based on Hi-Lo strategy"""
        if self.__suit == "ShuffleCard":
            return 0
        if isinstance(self.__value, int) and 2 <= self.__value <= 6:
            return 1  # Low cards increase the count
        elif isinstance(self.__value, int) and 7 <= self.__value <= 9:
            return 0  # Neutral cards don't affect the count
        else:  # 10, J, Q, K, A are high cards
            return -1  # High cards decrease the count
        
    def shuffle_card(self):
        return self.__suit == "ShuffleCard"

    def __str__(self):
        if self.shuffle_card():
            return "Shuffle Card"
        else:
            return str(self.__value) + " of " + self.__suit

    __repr__ = __str__

class Deck(Base):
    __suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
    __values = list(range(2, 11)) + ["Jack", "Queen", "King", "Ace"]

    def __init__(self, n_decks=6):
        Base.__init__(self)
        self.__n_decks = n_decks
        self.__cards = []
        self.reset()
            
    def reset(self):
        """Reset the deck with new cards and shuffle card"""
        self.__cards = []
        for _ in range(self.__n_decks):
            self.__cards.extend(self.__make_deck())
            
        # Add shuffle card at random position in the last quarter of the deck
        # This simulates a "cut card" or "plastic card" in casino play
        deck_size = len(self.__cards)
        position = random.randint(int(deck_size * 0.75), deck_size - 1)
        self.__cards.insert(position, Card("ShuffleCard"))
    
    def __make_deck(self):
        deck = []
        for suit in self.__suits:
            for value in self.__values:
                deck.append(Card(suit, value))
        return deck
    
    def shuffle(self):
        random.shuffle(self.__cards)
        
    def deal(self):
        if len(self.__cards) > 0:
            return self.__cards.pop()
        else:
            # If we run out of cards, reset and shuffle the deck
            self.reset()
            self.shuffle()
            self.message(self.INFO, "Deck has been reset and shuffled")
            return self.__cards.pop()
            
    def cards_remaining(self):
        return len(self.__cards)

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

Card Counter is out of chips!

Final chip counts:
Card Counter: 0 chips
Computer Player 1: 0 chips
Computer Player 2: 0 chips
Computer Player 3: 0 chips


8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 